In [ ]:
import sys
import os
import numpy as np
import cv2
from PIL import Image
from PIL.ImageQt import ImageQt
from PyQt5 import QtCore, QtGui, QtWidgets, uic
from PyQt5.QtWidgets import QWidget, QLabel
import pickle

In [ ]:
# Function to save data to pickle file
def save_pickle(filename, data):
    with open(filename, "wb") as fo:
        pickle.dump(data, fo, protocol=pickle.HIGHEST_PROTOCOL)

# Function to load data from pickle file
def load_pickle(filename):
    with open(filename, 'rb') as fo:
        return pickle.load(fo)

# Main application class
class GaitDemo(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()

        # Load the UI file
        uic.loadUi("stridesentinelUI.ui", self)

        # Set up UI elements
        self.showFullScreen()

        # Initialize camera
        self.capture = cv2.VideoCapture(0)  # Adjust camera index if necessary
        if not self.capture.isOpened():
            print("Error: Could not open camera.")
            sys.exit()

        # Initialize variables
        self.currentFrame = np.array([])
        self.firstFrame = None
        self.register_state = False
        self.recognition_state = False
        self.save_on = False
        self.gei_fix_num = 20

        # Connect UI buttons to methods
        self.save_gei.clicked.connect(self.save_gei_f)
        self.register_2.clicked.connect(self.register_show)
        self.recognize.clicked.connect(self.recognition_show)
        self.updater.clicked.connect(self.update_bk)

        # Load dataset or initialize if it doesn't exist
        self.load_dataset()

        # Setup timer for video stream
        self._timer = QtCore.QTimer(self)
        self._timer.timeout.connect(self.play)
        self._timer.start(27)  # Adjust timing as needed for your camera

        # Show the main window
        self.show()

    # Method to handle saving GEI
    def save_gei_f(self):
        self.save_on = True
        self.state_print.setPlainText('Saving!')

    # Method to handle registering
    def register_show(self):
        self.register_state = True
        self.recognition_state = False
        self.state_print.setPlainText('Register!')
        self.gei_current = np.zeros((128, 88), np.single)
        self.numInGEI = 0

    # Method to load dataset from pickle file or initialize if absent
    def load_dataset(self):
        self.data_path = './GaitData.pkl'
        if QtCore.QFile.exists(self.data_path):
            dic = load_pickle(self.data_path)
            self.num = dic['num']
            self.gei = dic['gei']
            self.name = dic['name']
        else:
            self.num = 0
            self.gei = np.zeros([100, 128, 88], np.uint8)
            self.name = []
            dic = {'num': self.num, 'gei': self.gei, 'name': self.name}
            save_pickle(self.data_path, dic)

        self.id_num.setPlainText('%d' % self.num)
        self.state_print.setPlainText('Running!')

    # Method to handle video stream processing
    def play(self):
        try:
            ret, frame = self.capture.read()
            if not ret:
                print("STRIDESENTINEL by ml mavericks.")
                return

            frame = cv2.resize(frame, (512, 384))
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0)

            if self.firstFrame is None:
                self.firstFrame = gray

            frameDelta = cv2.absdiff(self.firstFrame, gray)
            thresh = cv2.threshold(frameDelta, 50, 255, cv2.THRESH_BINARY)[1]
            self.FrameForUpdate = gray
            thresh = cv2.dilate(thresh, None, iterations=2)
            (cnts, _) = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            thresh = np.array(thresh)
            max_rec = 0

            for c in cnts:
                if cv2.contourArea(c) < 500:
                    continue
                (x, y, w, h) = cv2.boundingRect(c)

                if w > 25 and h > 50:
                    if max_rec < w * h:
                        max_rec = w * h
                        (x_max, y_max, w_max, h_max) = cv2.boundingRect(c)

            if max_rec > 0:
                cv2.rectangle(frame, (x_max, y_max), (x_max + w_max, y_max + h_max), (0, 255, 0), 2)
                if x_max > 20:
                    if self.register_state or self.recognition_state:
                        nim = np.zeros([thresh.shape[0] + 10, thresh.shape[1] + 10], np.single)
                        nim[y_max + 5:(y_max + h_max + 5), x_max + 5:(x_max + w_max + 5)] = thresh[y_max:(y_max + h_max), x_max:(x_max + w_max)]
                        offsetX = 20
                        ty, tx = (nim > 100).nonzero()
                        sy, ey = ty.min(), ty.max() + 1
                        sx, ex = tx.min(), tx.max() + 1
                        h = ey - sy
                        w = ex - sx

                        if h > w:
                            cx = int(tx.mean())
                            cenX = h / 2
                            start_w = int(cenX - (cx - sx))
                            if max(cx - sx, ex - cx) < cenX:
                                start_w = int(cenX - (cx - sx))  # Ensure integer
                            
                            tim = np.zeros((h, h), np.single)
                            tim[:, int(start_w):int(start_w + w)] = nim[int(sy):int(ey), int(sx):int(ex)]  # Ensure all indices are integers

                            rim = Image.fromarray(np.uint8(tim)).resize((128, 128), resample=Image.LANCZOS)
                            tim = np.array(rim)[:, int(offsetX):int(offsetX + 88)]  # Ensure all indices are integers

                            if self.numInGEI < self.gei_fix_num:
                                self.gei_current += tim
                            self.numInGEI += 1

                        if self.numInGEI > self.gei_fix_num:
                            if self.save_on:
                                self.gei[self.num, :, :] = self.gei_current / self.gei_fix_num
                                Image.fromarray(np.uint8(self.gei_current / self.gei_fix_num)).save('./gei/gei%02d%s.jpg' % (self.num, self.id_name.toPlainText()))
                                self.name.append(self.id_name.toPlainText())
                                self.num += 1
                                self.id_num.setPlainText('%d' % self.num)
                                dic = {'num': self.num, 'gei': self.gei, 'name': self.name}
                                save_pickle(self.data_path, dic)
                                self.save_on = False
                                self.state_print.setPlainText('Saved!')
                            elif self.recognition_state:
                                self.gei_query = self.gei_current / (self.gei_fix_num)
                                score = np.zeros(self.num)
                                self.gei_to_com = np.zeros([128, 88], np.single)
                                for q in range(self.num):
                                    self.gei_to_com = self.gei[q, :, :]
                                    score[q] = np.exp(-(((self.gei_query[:] - self.gei_to_com[:]) / (128 * 88)) ** 2).sum())

                                if score.size == 0:
                                    print("Error: The score array is empty.")
                                    return

                                q_id = score.argmax()
                                if True:
                                    id_rec = '%s' % self.name[q_id]
                                    cv2.putText(frame, id_rec, (x_max + 20, y_max + 20), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.0, thickness=2, color=(0, 0, 255))
            else:
                self.gei_current = np.zeros((128, 88), np.single)
                self.numInGEI = 0

            self.currentFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            self.seg = np.repeat(thresh[:, :, np.newaxis], 3, axis=2)
            self.seg[:, :, 0] = 0
            self.seg[:, :, 2] = 0
            self.display_video_stream()
            self.display_segmentation_stream()

        except Exception as e:
            print(f"Error: {e}")

    # Method to display video stream on label
    def display_video_stream(self):
        qimage = QtGui.QImage(self.currentFrame, self.currentFrame.shape[1], self.currentFrame.shape[0], QtGui.QImage.Format_RGB888)
        pix = QtGui.QPixmap.fromImage(qimage)
        self.video_label.setPixmap(pix)

    # Method to display segmentation stream on label
    def display_segmentation_stream(self):
        seg_frame = QtGui.QImage(self.seg, self.seg.shape[1], self.seg.shape[0], QtGui.QImage.Format_RGB888)
        pix = QtGui.QPixmap.fromImage(seg_frame)
        self.seg_label.setPixmap(pix)

    # Method to update background frame
    def update_bk(self):
        self.firstFrame = self.FrameForUpdate

    # Method to handle recognition
    def recognition_show(self):
        self.recognition_state = True
        self.register_state = False
        self.state_print.setPlainText('Recognize!')

    # Override closeEvent to properly release resources
    def closeEvent(self, event):
        self.capture.release()  # Release the camera
        event.accept()

# Main function to run the application
if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    window = GaitDemo()
    sys.exit(app.exec_())
